In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
locs = pd.read_csv('../Data/clean_data/location_codes.csv')

In [3]:
provs = pd.read_csv('../Data/clean_data/contagio_provincias.csv')

In [4]:
muns = pd.read_csv('../Data/clean_data/contagio_municipios.csv')

In [5]:
locs.head(1)

,com_id,prov_id,mun_id,NOMBRE
0,16,1,1051,Agurain/Salvatierra


In [7]:
provs.head(1)

,prov_id,km2,total_prov,densidad,R0,prob_inf
0,1,293470.0,336686.0,1.147259,5.736293,0.998614


In [8]:
muns.head(1)

,densidad,prov_id,mun_id,R0,prob_inf
0,1.491228,1,1001,7.45614,0.999985
